In [22]:
import pandas as pd
import io
import os

In [23]:
#Environment variables
WEATHER_DATA_DIR = 'Database/Weather Data'


# Function `split_csv_by_empty_row` to split a CSV file into two separate DataFrames based on an empty row that separates the tables in the file.

Here's how the code works:

1. It opens the CSV file specified by the `file_path` using the `open()` function and assigns it to the file object `f`.

2. It reads all the lines from the file using the `readlines()` method of the file object and assigns them to the list variable `lines`.

3. The code then searches for the empty row in the `lines` list. It does this by using a list comprehension and the `enumerate()` function to iterate over each line in the `lines` list. If a line is empty (after stripping any leading or trailing whitespace), its index is added to the `empty_row_indices` list.

4. Assuming there is only one empty row separating the tables in the CSV file, the code assigns the index of the first empty row to the variable `separator_index`.

5. The code then splits the lines into two separate lists based on the `separator_index`. The lines before the empty row are assigned to the list variable `table1_lines`, and the lines after the empty row (including the empty row itself) are assigned to the list variable `table2_lines`.

6. The code uses the `pd.read_csv()` function from the pandas library to convert the `table1_lines` and `table2_lines` lists into separate DataFrames. It does this by first joining the lines in each list into a single string using `''.join()`, and then passing the resulting string to `io.StringIO()` to create a file-like object that can be read by `pd.read_csv()`.

7. The resulting DataFrames are assigned to the variables `metadata_df` and `readings_df`.

8. Finally, the function returns the `metadata_df` and `readings_df` DataFrames as a tuple.

This function can be used to split a CSV file into two separate DataFrames based on an empty row, which can be useful for further processing or analysis of the data.

In [33]:
def split_csv_by_empty_row(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find the empty row
    empty_row_indices = [i for i, line in enumerate(lines) if line.strip() == '']

    # Assuming there's only one empty row separating the tables
    separator_index = empty_row_indices[0]

    # Split the lines into two lists based on the separator index
    table1_lines = lines[:separator_index]
    table2_lines = lines[separator_index + 1:]

    # Convert the lists of lines to DataFrames
    metadata_df = pd.read_csv(io.StringIO(''.join(table1_lines)))
    readings_df = pd.read_csv(io.StringIO(''.join(table2_lines)))

    return metadata_df, readings_df


## Splitting the CSV file into two DataFrames and merging them back together

1. Initializes an empty list called `all_dfs` which will be used to store datasets.

2. Loops through the files in the `WEATHER_DATA_DIR` directory.

3. Checks if the file has a `.csv` extension.

4. If the file is a `.csv` file, it proceeds to split the CSV file into two DataFrames using the `split_csv_by_empty_row()` function. The resulting DataFrames are assigned to `metadata_df` and `readings_df`.

5. Adds new columns ('latitude', 'longitude', 'elevation', 'timezone', 'timezone_abbreviation') to the `readings_df` DataFrame.

6. Merges the `metadata_df` and `readings_df` DataFrames based on the 'location_id' column using the `pd.merge()` function. The merged DataFrame is assigned to `merged_df`.

7. Drops redundant columns ('latitude_y', 'longitude_y', 'elevation_y', 'timezone_y', 'timezone_abbreviation_y') from the `merged_df` DataFrame using the `drop()` method.

8. Saves the merged DataFrame to a new CSV file in the `WEATHER_DATA_DIR` directory, overwriting the original file.

This code essentially processes each CSV file in the specified directory, adds new columns to the readings data, merges it with metadata, and saves the merged data back to the original file.

In [ ]:
# Empty list that will be populated with the datasets
all_dfs = []

# Loops through the files in the Datasets directory
for file in os.listdir(WEATHER_DATA_DIR):

    # Executes only the .csv files
    if(file.endswith('.csv')):

        # Split the CSV into two DataFrames
        metadata_df, readings_df = split_csv_by_empty_row(f'{WEATHER_DATA_DIR}/{file}')
        
        # Create new columns in readings_df
        readings_df['latitude'] = ''
        readings_df['longitude'] = ''
        readings_df['elevation'] = ''
        readings_df['timezone'] = ''
        readings_df['timezone_abbreviation'] = ''

        # Merge metadata_df and readings_df based on location_id of both DataFrames
        merged_df = pd.merge(metadata_df, readings_df, on='location_id')

        # Drop the reduncant columns
        merged_df.drop(columns=['latitude_y', 'longitude_y', 'elevation_y', 'timezone_y', 'timezone_abbreviation_y'], inplace=True)

        # Save the merged DataFrame to their respective csv file
        merged_df.to_csv(f'{WEATHER_DATA_DIR}/{file}', index=False)

        

## Dropping the Redundant fields and Renaming the fields

In [32]:
# Loops through the files in the Datasets directory
for file in os.listdir(WEATHER_DATA_DIR):
    # Read the csv file
    df = pd.read_csv(f'{WEATHER_DATA_DIR}/{file}')

    # Rename the columns
    df.rename(columns={'latitude_x': 'latitude', 'longitude_x': 'longitude', 'elevation_x': 'elevation', 'timezone_x': 'timezone', 'timezone_abbreviation_x': 'timezone_abbreviation'}, inplace=True)

    # Drop the reduncant columns
    df.drop(columns=['timezone', 'utc_offset_seconds'], inplace=True)

    # Save the DataFrame to their respective csv file
    df.to_csv(f'{WEATHER_DATA_DIR}/{file}', index=False)

